In [9]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

DATA_VERSION = "v3"

artists_data_path = f"../data/{DATA_VERSION}/artists.jsonl"
sessions_data_path = f"../data/{DATA_VERSION}/sessions.jsonl"
track_storage_data_path = f"../data/{DATA_VERSION}/track_storage.jsonl"
tracks_data_path = f"../data/{DATA_VERSION}/tracks.jsonl"
users_data_path = f"../data/{DATA_VERSION}/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Scalenie danych

In [10]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
# rename column
merged_data_frame = merged_data_frame.rename(columns={"name_x": "track_name"})
merged_data_frame = merged_data_frame.rename(columns={"name_y": "artist_name"})
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,mode,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_name,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-01 01:40:27.000,101,1lj8X10eP2DvdxbSWHu5dW,PLAY,124,1lj8X10eP2DvdxbSWHu5dW,Porta portese,15,230507,0,5zBAb4P2NoTps0shbgZvgj,1989-08-10,0.532,0.543,5,NaN,-12.081,0.156,0.420,0.0000,0.380,0.584,139.353,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,5zBAb4P2NoTps0shbgZvgj,Claudio Baglioni,"[classic italian pop, italian adult pop]"
1,2023-03-29 22:41:06.217,101,6OSi9aZITn8mDjTwf53OCY,PLAY,143,6OSi9aZITn8mDjTwf53OCY,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.176,0.122,0.0577,0.244,0.362,125.068,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,5zBAb4P2NoTps0shbgZvgj,Claudio Baglioni,"[classic italian pop, italian adult pop]"
2,2023-03-29 22:41:19.489,101,6OSi9aZITn8mDjTwf53OCY,SKIP,143,6OSi9aZITn8mDjTwf53OCY,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.176,0.122,0.0577,0.244,0.362,125.068,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,5zBAb4P2NoTps0shbgZvgj,Claudio Baglioni,"[classic italian pop, italian adult pop]"
3,2023-03-17 03:27:49.325,122,6OSi9aZITn8mDjTwf53OCY,PLAY,428,6OSi9aZITn8mDjTwf53OCY,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.176,0.122,0.0577,0.244,0.362,125.068,4,Jan Rudzik,Kraków,pl. Krokusowa 01,"[roots rock, latin rock, latin rock]",True,5zBAb4P2NoTps0shbgZvgj,Claudio Baglioni,"[classic italian pop, italian adult pop]"
4,2023-03-17 03:29:43.096,122,6OSi9aZITn8mDjTwf53OCY,SKIP,428,6OSi9aZITn8mDjTwf53OCY,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.176,0.122,0.0577,0.244,0.362,125.068,4,Jan Rudzik,Kraków,pl. Krokusowa 01,"[roots rock, latin rock, latin rock]",True,5zBAb4P2NoTps0shbgZvgj,Claudio Baglioni,"[classic italian pop, italian adult pop]"


## Wyliczenie etykiety, czy dana piosenka w danej sesji została pominięta przez danego użytkownika

In [11]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")



## Dodanie nowych atrybutów

In [12]:
merged_data_frame["number_of_matching_genres"] = merged_data_frame.apply(lambda x: len(set(x["genres"]).intersection(set(x["favourite_genres"]))), axis=1)
merged_data_frame["month"] = merged_data_frame["timestamp"].dt.month
merged_data_frame["day_of_week"] = merged_data_frame["timestamp"].dt.dayofweek
merged_data_frame["hour_of_day"] = merged_data_frame["timestamp"].dt.hour
merged_data_frame["genres_with_favourite_genres"] = merged_data_frame["genres"] + merged_data_frame["favourite_genres"]
merged_data_frame["common_genres"] = merged_data_frame["genres_with_favourite_genres"].apply(lambda x: list(set(x)))

## Usunięcie niepotrzebnych kolumn

In [13]:
merged_data_frame = merged_data_frame.drop(columns=["id_x", "id_y", "event_type"])
merged_data_frame.head(10)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,mode,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_name,city,street,favourite_genres,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-01 01:40:27.000,101,1lj8X10eP2DvdxbSWHu5dW,124,Porta portese,15,230507,0,5zBAb4P2NoTps0shbgZvgj,1989-08-10,0.532,0.543,5,NaN,-12.081,0.1560,0.420,0.000000,0.3800,0.584,139.353,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,Claudio Baglioni,"[classic italian pop, italian adult pop]",False,0,1,6,1,"[classic italian pop, italian adult pop, latin alternative, argentine rock, singer-songwriter]","[singer-songwriter, latin alternative, classic italian pop, argentine rock, italian adult pop]"
2,2023-03-29 22:41:19.489,101,6OSi9aZITn8mDjTwf53OCY,143,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.1760,0.122,0.057700,0.2440,0.362,125.068,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,Claudio Baglioni,"[classic italian pop, italian adult pop]",True,0,3,2,22,"[classic italian pop, italian adult pop, latin alternative, argentine rock, singer-songwriter]","[singer-songwriter, latin alternative, classic italian pop, argentine rock, italian adult pop]"
4,2023-03-17 03:29:43.096,122,6OSi9aZITn8mDjTwf53OCY,428,Cartolina Rosa,16,190000,0,5zBAb4P2NoTps0shbgZvgj,1972,0.381,0.650,9,NaN,-12.908,0.1760,0.122,0.057700,0.2440,0.362,125.068,4,Jan Rudzik,Kraków,pl. Krokusowa 01,"[roots rock, latin rock, latin rock]",True,Claudio Baglioni,"[classic italian pop, italian adult pop]",True,0,3,4,3,"[classic italian pop, italian adult pop, roots rock, latin rock, latin rock]","[latin rock, classic italian pop, italian adult pop, roots rock]"
5,2023-01-01 01:44:17.507,101,7yBAtD7gPsmcqVRTmVGsq0,124,Vamos A Hacernos Los Locos,21,211160,0,66U02qGDesTqZImnLSiYeE,2004-11-15,0.796,0.704,7,NaN,-4.208,0.0320,0.472,0.000000,0.0974,0.636,122.003,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,Elvis Martinez,"[bachata, bachata dominicana, dominican pop, latin, tropical]",False,0,1,6,1,"[bachata, bachata dominicana, dominican pop, latin, tropical, latin alternative, argentine rock, singer-songwriter]","[bachata dominicana, bachata, latin alternative, singer-songwriter, dominican pop, latin, tropical, argentine rock]"
7,2023-01-01 01:47:48.667,101,799JeG7U9d9wE00HPb2tvn,124,Paloma Ajena - En Vivo,41,223857,0,5ZAR5Usb11xw4KENbEE8uZ,2018-06-19,0.765,0.751,5,NaN,-7.853,0.0359,0.259,0.000909,0.2080,0.787,103.979,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,Agua Marina,"[cumbia peruana, pop peruano]",False,0,1,6,1,"[cumbia peruana, pop peruano, latin alternative, argentine rock, singer-songwriter]","[singer-songwriter, latin alternative, argentine rock, cumbia peruana, pop peruano]"
10,2023-01-01 01:54:14.075,101,7k5HKx34CmA45SA07LXHsk,124,Şakası Yok,31,264243,0,69wYy20lPWCHSgDIHKpWcD,2008,0.657,0.591,9,NaN,-8.715,0.0373,0.519,0.000986,0.1340,0.692,160.032,3,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,Nedim Zeper,[deep turkish pop],True,0,1,6,1,"[deep turkish pop, latin alternative, argentine rock, singer-songwriter]","[singer-songwriter, deep turkish pop, latin alternative, argentine rock]"
11,2023-01-01 01:54:23.075,101,6OhLezBLy0BblgdhnOfa7b,124,Syrgas (Till Anna),50,214572,0,2iUVnGKQclQfYjdHGSZcX0,2017-11-01,0.743,0.911,11,0.0,-3.146,0.1220,0.121,0.000000,0.3720,0.590,104.958,4,Kalina Irla,Poznań,pl. Zaciszna 65/75,"[latin alternative, argentine rock, singer-songwriter]",False,LBSB,[swedish pop],False,0,1,6,1,"[swedish pop, latin alternative, argentine rock, singe

In [14]:
merged_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11525 entries, 0 to 19351
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     11525 non-null  datetime64[ns]
 1   user_id                       11525 non-null  int64         
 2   track_id                      11525 non-null  object        
 3   session_id                    11525 non-null  int64         
 4   track_name                    11525 non-null  object        
 5   popularity                    11525 non-null  int64         
 6   duration_ms                   11525 non-null  int64         
 7   explicit                      11525 non-null  int64         
 8   id_artist                     11525 non-null  object        
 9   release_date                  11525 non-null  object        
 10  danceability                  11525 non-null  float64       
 11  energy                        115

In [15]:
merged_data_frame.describe()

,timestamp,user_id,session_id,popularity,duration_ms,explicit,danceability,energy,key,mode,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,number_of_matching_genres,month,day_of_week,hour_of_day
count,11525,11525.000000,11525.000000,11525.000000,1.152500e+04,11525.000000,11525.000000,11525.000000,11525.00000,2398.000000,11525.000000,11525.000000,11525.000000,11525.000000,11525.000000,11525.000000,11525.000000,11525.00000,11525.000000,11525.000000,11525.000000,11525.000000
mean,2023-02-16 15:48:42.742040064,125.530759,477.643384,45.968590,2.303387e+05,0.083731,0.564767,0.598746,5.28538,0.616347,-9.092024,0.084310,0.326333,0.071746,0.201627,0.538316,121.203084,3.93102,0.102126,2.108720,3.135792,11.092061
min,2023-01-01 01:40:27,101.000000,124.000000,0.000000,3.075400e+04,0.000000,0.000000,0.000431,0.00000,0.000000,-35.898000,0.000000,0.000001,0.000000,0.011100,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000
25%,2023-01-23 06:50:49.888000,113.000000,297.000000,26.000000,1.803480e+05,0.000000,0.454000,0.427000,2.00000,0.000000,-11.451000,0.033500,0.036100,0.000000,0.095000,0.334000,99.112000,4.00000,0.000000,1.000000,1.000000,5.000000
50%,2023-02-16 06:29:33.899000064,127.000000,483.000000,41.000000,2.150930e+05,0.000000,0.570000,0.620000,5.00000,1.000000,-8.313000,0.042800,0.221000,0.000049,0.125000,0.544000,120.232000,4.00000,0.000000,2.000000,3.000000,11.000000
75%,2023-03-12 02:59:40.728000,138.000000,650.000000,75.000000,2.622400e+05,0.000000,0.688000,0.789000,9.00000,1.000000,-6.050000,0.077500,0.584000,0.006020,0.246000,0.744000,138.113000,4.00000,0.000000,3.000000,5.000000,17.000000
max,2023-04-03 13:28:46.779000,150.000000,831.000000,96.000000,2.700449e+06,1.000000,0.982000,0.999000,11.00000,1.000000,1.107000,0.963000,0.996000,0.999000,0.996000,0.993000,211.545000,5.00000,2.000000,4.000000,6.000000,23.000000
std,NaN,14.399409,204.290069,25.326611,1.002691e+05,0.276996,0.165867,0.228953,3.55258,0.486376,4.313837,0.122285,0.310340,0.203803,0.185640,0.253893,28.781956,0.37818,0.314080,0.906549,2.050484,6.718114


## Współczynnik informacji wzajemnej

#### Pominięto atrybuty ciągłe
Sprawdzamy, czy atrybuty niosą ze sobą informację o zmiennej celu.

In [16]:
from sklearn import metrics
from scipy.stats import entropy

print("Entropy of skipped column: ", str(round(entropy(merged_data_frame["skipped"].value_counts()), 4)))

df = merged_data_frame.copy()
df = df.drop(columns=["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])
if DATA_VERSION == "v3":
  df = df.drop(columns=["mode", "key"])

df["favourite_genres"] = df["favourite_genres"].apply(lambda x: str(x))
df["genres"] = df["genres"].apply(lambda x: str(x))
df["genres_with_favourite_genres"] = df["genres_with_favourite_genres"].apply(lambda x: str(x))
df["common_genres"] = df["common_genres"].apply(lambda x: str(x))

df["timestamp"] = df["timestamp"].astype(str)

mutual_info_score = {}

for column in df.columns:
  if column != "skipped":
    mutual_info_score[column] = round(metrics.mutual_info_score(df["skipped"], df[column]), 4)

# print sorted by value
for key, value in sorted(mutual_info_score.items(), key=lambda item: item[1], reverse=True):
  print("%s: %s" % (key, value))

df.info()

Entropy of skipped column:  0.6442
timestamp: 0.5198
genres_with_favourite_genres: 0.4922
common_genres: 0.4919
track_id: 0.3194
track_name: 0.3161
duration_ms: 0.2861
id_artist: 0.2019
name: 0.2018
session_id: 0.1933
genres: 0.1649
release_date: 0.1604
user_id: 0.1569
artist_name: 0.1569
street: 0.1569
favourite_genres: 0.1569
city: 0.0267
popularity: 0.0127
premium_user: 0.0096
hour_of_day: 0.0019
day_of_week: 0.0017
number_of_matching_genres: 0.0016
explicit: 0.0015
time_signature: 0.0003
month: 0.0001
<class 'pandas.core.frame.DataFrame'>
Index: 11525 entries, 0 to 19351
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   timestamp                     11525 non-null  object
 1   user_id                       11525 non-null  int64 
 2   track_id                      11525 non-null  object
 3   session_id                    11525 non-null  int64 
 4   track_name                    11

## Macierz korelacji

#### Pominięto atrybuty dyskretne

In [17]:
df = merged_data_frame.copy()
df = df.drop(columns=["genres", "favourite_genres", "genres_with_favourite_genres", "common_genres", "track_id", "track_name", "id_artist", "release_date", "artist_name", "city", "street", "name"])

df["skipped"] = df["skipped"].astype(int)
df["premium_user"] = df["premium_user"].astype(int)
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")

correlation_matrix = df.corrwith(df["skipped"])
correlation_matrix.head(50)

timestamp                   -0.016237
user_id                     -0.000751
session_id                  -0.003183
popularity                  -0.097975
duration_ms                  0.072772
explicit                    -0.053877
danceability                -0.137410
energy                       0.021531
key                         -0.014548
mode                         0.035226
loudness                    -0.026727
speechiness                 -0.060179
acousticness                -0.024754
instrumentalness             0.036334
liveness                     0.027968
valence                     -0.039814
tempo                        0.030428
time_signature              -0.022626
premium_user                 0.139855
skipped                      1.000000
number_of_matching_genres   -0.021418
month                       -0.005247
day_of_week                  0.003861
hour_of_day                 -0.016083
dtype: float64

In [18]:
# delete unwanted columns
merged_data_frame = merged_data_frame.drop(columns=["timestamp", "user_id", "track_id", "session_id", "track_name", "popularity", "duration_ms", "explicit", "id_artist", "release_date", "danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "artist_name", "city", "street", "premium_user", "name", "number_of_matching_genres", "month", "day_of_week", "hour_of_day", "genres_with_favourite_genres", "common_genres"])

# save to jsonl file
merged_data_frame.to_json(f"../data/{DATA_VERSION}/merged_data.jsonl", orient="records", lines=True)

merged_data_frame.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11525 entries, 0 to 19351
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mode              2398 non-null   float64
 1   time_signature    11525 non-null  int64  
 2   favourite_genres  11525 non-null  object 
 3   genres            11525 non-null  object 
 4   skipped           11525 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 719.5+ KB


## Podsumowanie

Jak widać na podstawie współczyników informacji wzajemnej, połączenie atrybutów "genres" i "favourite_genres" jest dobrym atrybutem do uczenia naszego modelu predykcyjnego.
Innymi dobrymi atrybutami są: "tempo", "duration_ms" i "loudness".

Początkowo zdecydowaliśmy się na użycie atrybutów "genres" i "favourite_genres" jako danych wejściowych modelu. Zmienną celu jest etykieta "skipped".
